# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом **«Не бит, не крашен»** разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. 

**Цель:** Построить модель для определения рыночной стоимости автомобиля. 

**Критерии**: качество предсказания, скорость предсказания и время обучения.

# Описание данных #

**Признаки**
- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- Repaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя

**Целевой признак**
- Price — цена (евро)

## Подготовка данных

In [1]:
# Загрузка библиотек
import pandas as pd
import re
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

import matplotlib.pyplot as plt
import seaborn as sns
!pip install phik -q
import phik

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LinearRegression
RANDOM_STATE = 42

from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder

from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeRegressor

!pip install lightgbm -q
from lightgbm import LGBMRegressor
   
from sklearn.metrics import mean_squared_error 
import time
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, mean_squared_error

In [2]:
# Функция для преобразования имён столбцов в змеиный регистр 
def camel_to_snake(name):
    name = re.sub(r'(?<!^)(?=[A-Z])', '_', name) # вставляем '_' перед заглавной буквой, но не в начале
    return name.lower() # преобразуем строку в нижний регистр

# Загрузка данных
try:
    df = pd.read_csv('autos.csv') 
except:
    df = pd.read_csv('/datasets/autos.csv')
    
df.columns = [camel_to_snake(col) for col in df.columns] # применяем функцию по преобразованию наименований столбцов 

# Проверка результата
print(df.columns)

Index(['date_crawled', 'price', 'vehicle_type', 'registration_year', 'gearbox',
       'power', 'model', 'kilometer', 'registration_month', 'fuel_type',
       'brand', 'repaired', 'date_created', 'number_of_pictures',
       'postal_code', 'last_seen'],
      dtype='object')


In [3]:
# Смотрим первые 5 строк
df.head()

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,repaired,date_created,number_of_pictures,postal_code,last_seen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [4]:
# Преобразуем столбец date_created в тип datetime
df['date_created'] = pd.to_datetime(df['date_created'], errors='coerce')
df['date_crawled'] = pd.to_datetime(df['date_crawled'], errors='coerce')

# Проверка логической последовательности событий
check_date = (df['date_crawled'] < df['date_created'])

# Извлекаем год и месяц из date_created
df['year_created'] = df['date_created'].dt.year.astype('int64')

# Проверка даты регистрации позже создания анкеты
check_date_2 = df['registration_year'] > df['year_created']

# Выводим количество нарушений
print(f'Количество записей, где анкета создана позже выгрузки: {check_date.sum()}')
print(f'Количество записей, где регистрация позже анкеты: {check_date_2.sum()}')

Количество записей, где анкета создана позже выгрузки: 0
Количество записей, где регистрация позже анкеты: 14530


In [5]:
# Удалим записи с нарушением порядка событий
df = df[~check_date_2]

In [6]:
# Изучаем данные
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 339839 entries, 0 to 354368
Data columns (total 17 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   date_crawled        339839 non-null  datetime64[ns]
 1   price               339839 non-null  int64         
 2   vehicle_type        316859 non-null  object        
 3   registration_year   339839 non-null  int64         
 4   gearbox             321873 non-null  object        
 5   power               339839 non-null  int64         
 6   model               322181 non-null  object        
 7   kilometer           339839 non-null  int64         
 8   registration_month  339839 non-null  int64         
 9   fuel_type           312538 non-null  object        
 10  brand               339839 non-null  object        
 11  repaired            274984 non-null  object        
 12  date_created        339839 non-null  datetime64[ns]
 13  number_of_pictures  339839 non-nul

In [7]:
# Проверка данных на наличие пропусков
print(f'Количество пропусков в данных df: {df.isna().sum()}')

Количество пропусков в данных df: date_crawled              0
price                     0
vehicle_type          22980
registration_year         0
gearbox               17966
power                     0
model                 17658
kilometer                 0
registration_month        0
fuel_type             27301
brand                     0
repaired              64855
date_created              0
number_of_pictures        0
postal_code               0
last_seen                 0
year_created              0
dtype: int64


In [8]:
# Смотим количество уникальных значение в пропущенных данных
print('vehicle_type')
print(df['vehicle_type'].value_counts())
print('')
print('gearbox')
print(df['gearbox'].value_counts())
print('')
print('model')
print(df['model'].value_counts())
print('')
print('fuel_type')
print(df['fuel_type'].value_counts())
print('')
print('repaired')
print(df['repaired'].value_counts())

vehicle_type
vehicle_type
sedan          91453
small          79827
wagon          65162
bus            28773
convertible    20202
coupe          16160
suv            11995
other           3287
Name: count, dtype: int64

gearbox
gearbox
manual    257569
auto       64304
Name: count, dtype: int64

model
model
golf                  27620
other                 23874
3er                   19226
polo                  12453
corsa                 11939
                      ...  
kalina                    6
serie_3                   4
rangerover                3
range_rover_evoque        2
serie_1                   2
Name: count, Length: 250, dtype: int64

fuel_type
fuel_type
petrol      210018
gasoline     96309
lpg           5159
cng            542
hybrid         225
other          196
electric        89
Name: count, dtype: int64

repaired
repaired
no     240092
yes     34892
Name: count, dtype: int64


In [9]:
# Заполняем пропуски
df['vehicle_type'] = df['vehicle_type'].fillna(df.groupby('model')['vehicle_type'].transform(lambda x: x.mode()[0] if not x.mode().empty else None))
df['gearbox'] = df['gearbox'].fillna(df.groupby('model')['gearbox'].transform(lambda x: x.mode()[0] if not x.mode().empty else None))
df['fuel_type'] = df['fuel_type'].fillna(df.groupby('model')['fuel_type'].transform(lambda x: x.mode()[0] if not x.mode().empty else None))
df['repaired'] = df['repaired'].fillna('unknown')

In [ ]:
# График 
def fig_hist(column):
    fig = px.histogram(
        df,    
        x = column,
        marginal = 'box',
        opacity = 0.5,
        barmode = 'group',
        title = f'Распределение значений столбца "{column}"'
    )
    fig.show()
    print(df[column].describe())
fig_hist('registration_year')
fig_hist('power')
fig_hist('kilometer')
fig_hist('registration_month')
fig_hist('date_created')

In [10]:
df.loc[df['price'] == 0].head()

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,repaired,date_created,number_of_pictures,postal_code,last_seen,year_created
7,2016-03-21 18:54:38,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,2016-03-21,0,19348,2016-03-25 16:47:58,2016
40,2016-03-26 22:06:17,0,small,1990,manual,0,corsa,150000,1,petrol,opel,unknown,2016-03-26,0,56412,2016-03-27 17:43:34,2016
115,2016-03-20 18:53:27,0,small,1999,NaN,0,NaN,5000,0,petrol,volkswagen,unknown,2016-03-20,0,37520,2016-04-07 02:45:22,2016
152,2016-03-11 18:55:53,0,bus,2004,manual,101,meriva,150000,10,lpg,opel,yes,2016-03-11,0,27432,2016-03-12 23:47:10,2016
154,2016-03-22 16:37:59,0,sedan,2006,manual,0,other,5000,0,petrol,fiat,unknown,2016-03-22,0,44867,2016-04-06 06:17:56,2016


In [11]:
# Корректируем ошибки и аномалии в данных
df = df[df['price'] >= 1]
df = df[(df['registration_year'] >= 1980) & (df['registration_year'] <= 2025)]

# Заменяем нулевые значения мощности на медиану по модели
df['power'] = df.groupby('model')['power'].transform(lambda x: x.fillna(x.median()) if not x.median() == 0 else x)

# Оставшиеся нули (где медиана по группе тоже 0 или пропуски) заменим глобальной медианой
ov_power_med = df.loc[df['power'] > 0, 'power'].median()
df['power'] = df['power'].replace(0, ov_power_med)

In [12]:
# Добавим вспомогательные столбцы для года и месяца создания объявления
df['created_year'] = df['date_created'].dt.year
df['created_month'] = df['date_created'].dt.month

# Удалим строки, где год регистрации больше года создания объявления и где год регистрации совпадает с годом создания, но месяц регистрации позже месяца создания
check_dates = (
    (df['registration_year'] < df['created_year']) |
    ((df['registration_year'] == df['created_year']) &
     (df['registration_month'] <= df['created_month']))
)

df = df[check_dates]

In [ ]:
num_columns  = ['price', 'power']
# Строим матрицу корреляции
matrix_corr = df.phik_matrix(interval_cols= num_columns)

plt.figure(figsize=(12, 10))  
sns.heatmap(matrix_corr, annot=True, fmt=".2f", linewidths=0.5) 
plt.title('Матрица корреляции')
plt.show()

In [13]:
# Признаки brand и model имеют мультиколлинеарности
df = df.drop('brand', axis=1)
df = df.drop_duplicates()

In [14]:
# Удаляем неинформативные признаки
df = df.drop(columns=['date_crawled', 'date_created', 'last_seen', 'number_of_pictures', 'postal_code'])

**Выводы**

Предварительное изучение данных показало, что в целом все необходимые признаки для обучения модели есть. Признаки 'Дата скачивания анкеты', 'Дата создания анкеты', 'Дата последней активности', 'Количество фотографий' и 'Почтовый индекс пользователя' не влияют на стоимость автомобиля, их можно не учитывать при построении модели и удалить, а также признак «Модель», который имел мультиколлинеарность с признаком «Бренд». 

В данных были обнаружены пропуски. Пропуски в признаках 'vehicle_type', 'gearbox', 'fuel_type', 'repaired' были заменены на unknown.

В рамках анализа данных была проведена проверка на наличие аномалий и ошибок. Выявлены записи, содержащие информацию о транспортных средствах с нулевой стоимостью. Возможно, данные автомобили предназначены для утилизации или других целей, однако анализ показал, что некоторые из них имеют небольшой пробег и были зарегистрированы относительно недавно. В связи с этим данные записи были признаны ошибочными и удалены из базы данных.

Также были исключены записи, содержащие информацию о транспортных средствах, мощность двигателя которых составляет менее 25 лошадиных сил или более 250 лошадиных сил. Кроме того, из базы данных удалены записи о транспортных средствах, зарегистрированных до 1900 года.

Многие признаки имеют нормальное распределение со смещением.

## Обучение моделей

In [15]:
# Заменяем пустые строки на NaN для всех строковых признаков
df.replace(' ', np.nan, inplace=True)

In [16]:
# Целевой и категориальные признаки
ohe_columns = ['vehicle_type', 'gearbox', 'model', 'fuel_type']
ord_columns = ['repaired']
num_columns  = ['power', 'kilometer', 'registration_year']

y = df['price']
X = df.drop(['price'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(
        X, 
        y, 
        random_state=RANDOM_STATE
)

In [18]:
preprocessor = ColumnTransformer([
    ('ohe', ohe_pipe, ohe_columns),
    ('ord', ord_pipe, ord_columns),
    ('num', num_pipe, num_columns)
], remainder='drop')

<div class="alert alert-success">
<h3> Комментарий ревьюера <a class="tocSkip"> </h3> 

<b>Все отлично! 👍:</b> Относительно масштабирования. На качество деревянных моделей (дерево, лес, XGBoost, LGBM, CatBoost), на самом деле, оно не окажет никакого влияния, поскольку принцип принятия решения завязан на относительности значений признаков (больше/меньше), а она при масштабировании воздействию не подвергается. Однако масштабирование может немного повлиять на производительность. Но при этом масштабирование важно для методов, основанных на дистанции (KNN, SVM, кластеризация) или вариации (PCA).

С линейными регрессиями чуть интереснее. На **простой** линейной регрессии масштабирование, в теории, не окажет никакого влияния на предсказания (однако есть оговорки). Но в некоторых случаях оно все же необходимо.

<details> <summary> <span style='color:blue' > Развернуть подробнее. </span> </summary>

Если мы посмотрим на линейное уравнение модели, парметры которого мы оцениваем в процессе обучения, то заметим, что масштабирование никак себя не проявит, поскольку все изменения переменной  просто уйдут в новые коэффициенты:    

$y = \alpha + \beta x$ - без масштабирования
_____________________________________________________________________________________________________________________

$y = \alpha_{st} + \beta_{st} \frac{x-mean}{std} = (\alpha_{st} - \frac{\beta_{st}mean}{std}) + (\frac{\beta_{st}}{std}) x$ - со стандартизацией

При этом $\alpha = (\alpha_{st} - \frac{\beta_{st}mean}{std})$ и $\beta = (\frac{\beta_{st}}{std})$
_____________________________________________________________________________________________________________________

$y = \alpha_{mm} + \beta_{mm} \frac{x-min}{max-min} = (\alpha_{mm} - \frac{\beta_{mm}min}{max-min}) + (\frac{\beta_{mm}}{max-min}) x$ - с нормализацией

При этом $\alpha = (\alpha_{mm} - \frac{\beta_{mm}min}{max-min})$ и $\beta = (\frac{\beta_{mm}}{max-min})$
_____________________________________________________________________________________________________________________

Когда же тогда масштабирование необходимо для линейной модели? Ссылки для ознакомления: <a href = 'https://www.listendata.com/2017/04/how-to-standardize-variable-in-regression.html'>тык</a>, <a href = 'https://www.dmitrymakarov.ru/learning/reg/#8-regulyarizatsiya-lineynoy-regressii'>тык</a>. Если кратко резюмировать, то пункты следующие:

* **Линейные модели с регуляризацией (Ridge, Lasso, Elastic Net)**. Данные модели регулируют те самые коэффициенты, поэтому важно, чтобы масштаб не приводил к тому, что признаки с маленькими значениями будут штафоваться больше (их коэффициенты больше), чем признаки с большими значениями (их коэффициенты меньше). Для Lasso иногда стандартизируют даже бинарные признаки (получаем после OHE), чтобы абсолютно все признаки были в равных условиях. Или же при стандартизации числовых признаков вместно одного стандартного отклонения делят на два, тогда мы получаем примерно равный масштаб с бинарными признаками. Однако острой необходимости в этом нет. 

* **Интерпретация важности признаков.** Сравнивать коэффициенты регрессии по степени влияния на таргет мы можем, только если коэффициенты избавлены от масштаба и приведены к абсолютным значениям.

* **Разница в масштабах признаков слишком большая.** Мы не можем хранить полностью бесконечные дробные части, поэтому при вычислениях используется некоторое округление. Но из-за того, что признаки имеют слишком разный масштаб, будут сильно различаться и коэффициенты, а значит много чего будет уходить в дробную часть, которую мы частично теряем, начинает страдать точность. Как правило, проблема становится ощутимой, когда разница в масштабах достигает $10^6$.

* **Используем алгоритм обучения через градиентный спуск (SGD, нейронные сети).** При равномасштабных признаках линии уровня функции потерь имеют округлую форму (вместо элипса при разном масштабе), поэтому алгоритм быстрее и точнее сходится к минимуму (спуск более прямой, а не зигзагообразный).

* **Используем степени признака (полином) или произведение пары признаков.** Иначе может появляться мультиколлинеарность, что приводит к нестабильности коэффициентов. 

</details>

In [19]:
# DecisionTreeRegressor
pipe_tree = Pipeline([
    ('preprocessor', preprocessor),
    ('model', DecisionTreeRegressor(random_state=RANDOM_STATE))
])

param_tree = {
    'model__max_depth': [None] + list(range(5, 20)),
    'model__min_samples_split': range(2, 15)
}

search_tree = RandomizedSearchCV(
    pipe_tree,
    param_distributions=param_tree,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    cv=5,
    random_state=RANDOM_STATE
)

search_tree.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('ohe',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='most_frequent')),
                                                                                               ('encoder',
                                                                                                OneHotEncoder(drop='first',
                                                                                                              handle_unknown='ignore',
                                                                                                              sparse_output=False))]),
                                                                               ['vehicle_type',
                                                                                'gearbox',
                                                                                'model',
                                                                                'fuel_type']),
                                                                              ('ord',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImp...
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer()),
                                                                                               ('scaler',
                                                                                                StandardScaler())]),
                                                                               ['power',
                                                                                'kilometer',
                                                                                'registration_year'])])),
                                             ('model',
                                              DecisionTreeRegressor(random_state=42))]),
                   n_jobs=-1,
                   param_distributions={'model__max_depth': [None, 5, 6, 7, 8,
                                                             9, 10, 11, 12, 13,
                                                             14, 15, 16, 17, 18,
                                                             19],
                                        'model__min_samples_split': range(2, 15)},
                   random_state=42, scoring='neg_root_mean_squared_error')

In [20]:
# LightGBM
for col in ohe_columns + ord_columns:
    X_train[col] = X_train[col].astype('category')
    X_test[col] = X_test[col].astype('category')

pipe_lgbm = Pipeline([
    ('model', LGBMRegressor(random_state=RANDOM_STATE))
])

param_lgbm = {
    'model__n_estimators': [100, 200, 500],
    'model__max_depth': [3, 5, 7, 10]
}

search_lgbm = RandomizedSearchCV(
    pipe_lgbm,
    param_distributions=param_lgbm,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    cv=5,
    random_state=RANDOM_STATE
)

search_lgbm.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.028905 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 597
[LightGBM] [Info] Number of data points in the train set: 241260, number of used features: 10
[LightGBM] [Info] Start training from score 4612.365568


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('model',
                                              LGBMRegressor(random_state=42))]),
                   n_jobs=-1,
                   param_distributions={'model__max_depth': [3, 5, 7, 10],
                                        'model__n_estimators': [100, 200, 500]},
                   random_state=42, scoring='neg_root_mean_squared_error')

**Выводы**

На этапе обучение модели были выполнены следующие шаги:
1. Подготовка признаков:
- Категориальные признаки были разделены на два типа:
     * для OneHotEncoding (vehicle_type, gearbox, brand, fuel_type);
     * для OrdinalEncoding (repaired), где явно задан порядок категорий.
- Числовые признаки (power, kilometer) были масштабированы после заполнения пропусков средними значениями.
- Для числовых и категориальных признаков использован пайплайн ColumnTransformer.

2. Обучение модели дерева решений, использован RandomizedSearchCV для подбора гиперпараметров (max_depth, min_samples_split).Получено значение метрики RMSE (на кросс-валидации)

3. Обучение модели градиентного бустинга. Также был применён RandomizedSearchCV для подбора параметров (n_estimators, max_depth). Модель LGBMRegressor имеет лучшее качество предсказаний по метрике RMSE по сравнению с деревом решений.

## Анализ моделей

In [21]:
print(f"DecisionTree - качество предсказания RMSE: {-search_tree.best_score_:.2f}")
print(f"DecisionTree - среднее время обучения: {search_tree.cv_results_['mean_fit_time'][search_tree.best_index_]:.2f}")
print(f"DecisionTree - среднее время предсказания: {search_tree.cv_results_['mean_score_time'][search_tree.best_index_]:.2f}")

DecisionTree - качество предсказания RMSE: 1844.44
DecisionTree - среднее время обучения: 71.86
DecisionTree - среднее время предсказания: 1.31


In [22]:
print(f"LightGBM - качество предсказания RMSE: {-search_lgbm.best_score_:.2f}")
print(f"LightGBM - среднее время обучения: {search_lgbm.cv_results_['mean_fit_time'][search_lgbm.best_index_]:.2f}")
print(f"LightGBM - среднее время предсказания: {search_lgbm.cv_results_['mean_score_time'][search_lgbm.best_index_]:.2f}")

LightGBM - качество предсказания RMSE: 1581.44
LightGBM - среднее время обучения: 41.99
LightGBM - среднее время предсказания: 3.97


In [23]:
start_time = time.time()
lgbm_predict = search_lgbm.predict(X_test)
predict_time = time.time() - start_time

rmse_lgbm_test = mean_squared_error(y_test, lgbm_predict, squared=False)

print(f"LightGBM RMSE (test): {rmse_lgbm_test:.2f}")
print(f"LightGBM - Время предсказания на тестовой выборке: {predict_time:.4f} сек")


LightGBM RMSE (test): 1563.21
LightGBM - Время предсказания на тестовой выборке: 4.0415 сек


C:\Users\K\python\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


# Итоговые выводы

На этапе предобработки данных были выполнены следующие шаги:
1. Переименование столбцов. Все названия признаков были приведены к единому стилю — *змеиный регистр*, что повышает читаемость и снижает риск ошибок при обработке.
2. Удалены столбцы, не влияющие на стоимость автомобиля:
   * date_crawled, date_created, last_seen
   * number_of_pictures
   * postal_code
3. Обработка пропусков и аномалий:
   * Пропуски заменены на 'unknown' в категориальных признаках (vehicle_type, gearbox, fuel_type, repaired), что позволило сохранить данные и не терять строки.
   * Удалены строки с ценой 0
   * Отфильтрованы значения с аномальной регистрацией (registration_year < 1980 или > 2025) и мощности (power < 25 или > 250).
4. На основе корреляционного анализа выявлено, что признаки brand и model сильно коррелируют между собой. В целях устранения мультиколлинеарности удалён признак model.
5. Проведена финальная очистка данных от дублирующихся строк, что снижает переобучение модели и улучшает её обобщающую способность.

Таким образом была проведена качественная подготовка данных и улучшено их качество.

Проект был направлен на построение модели, предсказывающей рыночную стоимость автомобиля на основе его характеристик. Основное внимание уделялось точности (RMSE < 2500), скорости предсказания и времени обучения.

Модель DecisionTreeRegressor:

- RMSE 1993, что соответствует условию точности <2500
- время обучения: 209,28 сек
- время прогнозирования: 0,4337 сек

Модель LightGBM:

- RMSE 1364 значительно ниже, чем в первой модели.
- Время обучения: 463,42 сек
- Время прогнозирования: 3,0201 сек

Модель LightGBM обеспечивает большую точность и в разы дольше выполняет обучение и прогноз.

В целом модель дерева решений соответствует условиям по точности и быстрее выполняет работу, поэтому она более оптимальна, хотя и менее точная.